In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [21]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [22]:
ps.head()

,Date,User,Rig #,File,Ephys,Trans,Morph,Pilot,Pilot Details,cell type,...,Plate,Comments,Stacis call for reconstructions,Stacis call for class,Confidence,Bad dates,Notes.1,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,160318,P5,NaN,Ndnf - brl_160318_01,x,NaN,n,NaN,NaN,tdt+,...,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,234626,NaN
1,160318,P5,NaN,Ndnf - brl_160318_02,x,NaN,n,NaN,NaN,tdt+,...,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,234626,NaN
2,160318,P5,NaN,Ndnf - brl_160318_03,x,NaN,y,NaN,NaN,tdt+,...,NaN,NaN,F,aspiny,NaN,NaN,NaN,NaN,234626,NaN
3,160318,P5,NaN,Ndnf - brl_160318_04,x,x,y,NaN,NaN,tdt+,...,NaN,NaN,F,aspiny,NaN,NaN,0,NaN,234626,NaN
4,160318,P5,NaN,Ndnf - brl_160318_05,x,NaN,n,NaN,NaN,tdt+,...,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,234626,NaN


In [25]:
ps = ps[["Date", "User", "Post patch?", "Notes", "PCR cycles", "SM_QC_PF", "Resolution index", "Bad dates"]]
ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["Bad dates"] != 'x']
ps = ps[ps["SM_QC_PF"] != "cnt"]
#ps = ps[ps["PCR cycles"] == 21]
#ps = ps.drop('Bad dates', 1)
ps["PCR cycles"].value_counts()


21      891
18      470
pass      5
Name: PCR cycles, dtype: int64